# OFA with Transformer

## Preparation

### Google drive

In [ ]:
from google.colab import drive
import os
import shutil
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
import os
import shutil
drive.mount("/content/gdrive")

%cd /content/gdrive/My Drive/OFA-sys

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/OFA-sys


### fetching files

In [ ]:
!git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git
!pip install OFA/transformers/
!git lfs install
!git clone https://huggingface.co/OFA-Sys/OFA-base

Cloning into 'OFA'...
remote: Enumerating objects: 5696, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 5696 (delta 26), reused 24 (delta 15), pack-reused 5648
Receiving objects: 100% (5696/5696), 97.85 MiB | 15.09 MiB/s, done.
Resolving deltas: 100% (2105/2105), done.
Checking out files: 100% (3223/3223), done.


## Model

In [ ]:
%cd /content/gdrive/My Drive/OFA-sys
!pip install OFA/transformers/

from PIL import Image
from torchvision import transforms
from transformers import OFATokenizer, OFAModel
from transformers.models.ofa.generate import sequence_generator

mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 256

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
    transforms.ToTensor(), 
    transforms.Normalize(mean=mean, std=std)
])

ckpt_dir='./OFA-base'
tokenizer = OFATokenizer.from_pretrained(ckpt_dir)

/content/gdrive/My Drive/OFA-sys
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./OFA/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 120 kB 4.1 MB/s 
     |████████████████████████████████| 880 kB 42.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.1 MB/s 
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3916733 sha256=2e839469edce40fb94331d15d8076e45fb62ba

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


./OFA-base
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


## Test

In [ ]:
txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids
!wget http://farm4.staticflickr.com/3539/3836680545_2ccb331621_z.jpg 
!mv 3836680545_2ccb331621_z.jpg  test.jpg
img = Image.open('./test.jpg')
patch_img = patch_resize_transform(img).unsqueeze(0)

model = OFAModel.from_pretrained(ckpt_dir, use_cache=False)

--2022-09-15 13:18:10--  http://farm4.staticflickr.com/3539/3836680545_2ccb331621_z.jpg
Resolving farm4.staticflickr.com (farm4.staticflickr.com)... 52.84.159.78, 2600:9000:2377:2000:0:5a51:64c9:c681, 2600:9000:2377:9a00:0:5a51:64c9:c681, ...
Connecting to farm4.staticflickr.com (farm4.staticflickr.com)|52.84.159.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm4.staticflickr.com/3539/3836680545_2ccb331621_z.jpg [following]
--2022-09-15 13:18:10--  https://farm4.staticflickr.com/3539/3836680545_2ccb331621_z.jpg
Connecting to farm4.staticflickr.com (farm4.staticflickr.com)|52.84.159.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘3836680545_2ccb331621_z.jpg’

3836680545_2ccb3316     [ <=>                ] 131.20K  --.-KB/s    in 0.03s   

2022-09-15 13:18:11 (4.38 MB/s) - ‘3836680545_2ccb331621_z.jpg’ saved [134347]



In [ ]:
# ! pip install wget
import wget
import os

url = "https://images.saatchiart.com/saatchi/1798153/art/8560318/7624146-CBEYTUHG-7.jpg"
def add_Test(url, out_path = "."):
  
    wget.download(url, out = out_path)

dir_path = r'./testImg'
cnt = len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))])
add_Test(url, f'./testImg/test{cnt + 1}.jpg')

#https://github.com/minyong-jeong/minyong-jeong.github.io/raw/master/images/ryan.jpg
#http://farm4.staticflickr.com/3539/3836680545_2ccb331621_z.jpg 
#https://www.cleverfiles.com/howto/wp-content/uploads/2018/03/minion.jpg
#https://image.shutterstock.com/image-vector/architecture-sketch-drawing-buildingcity-600w-179448743.jpg
#https://images.saatchiart.com/saatchi/1798153/art/8560318/7624146-CBEYTUHG-7.jpg
#https://c8.alamy.com/zooms/9/ece0745d430e41f4a45e947817e9285d/w8tgbd.jpg
#https://easydrawingart.com/wp-content/uploads/2019/07/how-to-easily-draw-a-hammer.jpg
#https://en.pimg.jp/082/987/446/1/82987446.jpg

## Choice of Generators

In [ ]:
generator = sequence_generator.SequenceGenerator(
    tokenizer=tokenizer,
    beam_size=5,
    max_len_b=16,
    min_len=0,
    no_repeat_ngram_size=3,
)

import torch
data = {}
data["net_input"] = {"input_ids": inputs, 'patch_images': patch_img, 'patch_masks':torch.tensor([True])}

#gen_output = generator.generate([model], data)
#gen = [gen_output[i][0]["tokens"] for i in range(len(gen_output))]

#display(img)
#print(tokenizer.batch_decode(gen, skip_special_tokens=True)[0].strip())

gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3) # using transformer

display(img)
print(tokenizer.batch_decode(gen, skip_special_tokens=True)[0].strip())

In [ ]:
gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3) # using transformer